In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('wgan')):
    os.mkdir('saved_models_{}'.format('wgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('wgan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
import os
import sys

import numpy as np


class WGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
                            optimizer=optimizer,
                            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
                              optimizer=optimizer,
                              metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # ---------------------
                    #  Train Discriminator
                    # ---------------------

                    # Select a random batch of images
                    # idx = np.random.randint(0, X_train.shape[0], batch_size)
                    # imgs = X_train[idx]

                    # Sample noise as generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                    # Generate a batch of new images
                    gen_imgs = self.generator.predict(noise)

                    # Train the critic
                    d_loss_real = self.critic.train_on_batch(imgs, valid)
                    d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                    # Clip critic weights
                    for l in self.critic.layers:
                        weights = l.get_weights()
                        weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                        l.set_weights(weights)

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f] [G loss: %f]" % (epoch, global_step, 1 - d_loss[0], 1 - g_loss[0]))

                if global_step % sample_interval == 0:
                    self.mode_drop(epoch, global_step)
                    

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')

if __name__ == '__main__':
    wgan = WGAN()
    wgan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 16)        448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:5[D loss: 0.999912] [G loss: 1.000182]
epoch:0 step:10[D loss: 0.999919] [G loss: 1.000189]
epoch:0 step:15[D loss: 0.999920] [G loss: 1.000186]
epoch:0 step:20[D loss: 0.999918] [G loss: 1.000184]
epoch:0 step:25[D loss: 0.999916] [G loss: 1.000179]
epoch:0 step:30[D loss: 0.999920] [G loss: 1.000181]
epoch:0 step:35[D loss: 0.999919] [G loss: 1.000185]
epoch:0 step:40[D loss: 0.999919] [G loss: 1.000189]
epoch:0 step:45[D loss: 0.999920] [G loss: 1.000188]
epoch:0 step:50[D loss: 0.999918] [G loss: 1.000184]
epoch:0 step:55[D loss: 0.999922] [G loss: 1.000185]
epoch:0 step:60[D loss: 0.999923] [G loss: 1.000179]
epoch:0 step:65[D loss: 0.999923] [G loss: 1.000177]
epoch:0 step:70[D loss: 0.999924] [G loss: 1.000170]
epoch:0 step:75[D loss: 0.999927] [G loss: 1.000164]
epoch:0 step:80[D loss: 0.999924] [G loss: 1.000152]
epoch:0 step:85[D loss: 0.999930] [G loss: 1.000142]
epoch:0 step:90[D loss: 0.999935] [G loss: 1.000135]
epoch:0 step:95[D loss: 0.999937] [G loss: 1.00

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

epoch:49 step:192085[D loss: 1.000001] [G loss: 1.000038]
epoch:49 step:192090[D loss: 0.999797] [G loss: 1.000558]
epoch:49 step:192095[D loss: 1.000079] [G loss: 0.999785]
epoch:49 step:192100[D loss: 1.000239] [G loss: 1.000111]
epoch:49 step:192105[D loss: 1.000380] [G loss: 0.999710]
epoch:49 step:192110[D loss: 1.000064] [G loss: 1.000043]
epoch:49 step:192115[D loss: 0.999976] [G loss: 1.000045]
epoch:49 step:192120[D loss: 0.999948] [G loss: 0.999969]
epoch:49 step:192125[D loss: 0.999872] [G loss: 1.000231]
epoch:49 step:192130[D loss: 0.999855] [G loss: 1.000378]
epoch:49 step:192135[D loss: 0.999981] [G loss: 1.000208]
epoch:49 step:192140[D loss: 0.999845] [G loss: 1.000335]
epoch:49 step:192145[D loss: 0.999945] [G loss: 1.000205]
epoch:49 step:192150[D loss: 1.000136] [G loss: 0.999786]
epoch:49 step:192155[D loss: 0.999956] [G loss: 1.000080]
epoch:49 step:192160[D loss: 0.999943] [G loss: 0.999914]
epoch:49 step:192165[D loss: 1.000392] [G loss: 0.999609]
epoch:49 step:

epoch:49 step:192795[D loss: 0.999969] [G loss: 1.000156]
epoch:49 step:192800[D loss: 0.999894] [G loss: 1.000284]
epoch:49 step:192805[D loss: 0.999929] [G loss: 1.000159]
epoch:49 step:192810[D loss: 0.999949] [G loss: 1.000104]
epoch:49 step:192815[D loss: 0.999976] [G loss: 1.000097]
epoch:49 step:192820[D loss: 0.999987] [G loss: 1.000045]
epoch:49 step:192825[D loss: 0.999975] [G loss: 1.000036]
epoch:49 step:192830[D loss: 1.000043] [G loss: 0.999846]
epoch:49 step:192835[D loss: 1.000039] [G loss: 0.999959]
epoch:49 step:192840[D loss: 1.000020] [G loss: 0.999892]
epoch:49 step:192845[D loss: 1.000001] [G loss: 1.000077]
epoch:49 step:192850[D loss: 1.000235] [G loss: 0.999844]
epoch:49 step:192855[D loss: 1.000013] [G loss: 0.999849]
epoch:49 step:192860[D loss: 0.999839] [G loss: 0.999982]
epoch:49 step:192865[D loss: 1.000130] [G loss: 0.999738]
epoch:49 step:192870[D loss: 1.000440] [G loss: 0.999927]
epoch:49 step:192875[D loss: 0.999654] [G loss: 1.000554]
epoch:49 step:

epoch:49 step:193505[D loss: 0.999976] [G loss: 0.999987]
epoch:49 step:193510[D loss: 0.999959] [G loss: 1.000039]
epoch:49 step:193515[D loss: 1.000006] [G loss: 1.000146]
epoch:49 step:193520[D loss: 0.999930] [G loss: 1.000214]
epoch:49 step:193525[D loss: 0.999800] [G loss: 1.000141]
epoch:49 step:193530[D loss: 0.999949] [G loss: 1.000040]
epoch:49 step:193535[D loss: 1.000051] [G loss: 0.999984]
epoch:49 step:193540[D loss: 0.999971] [G loss: 0.999916]
epoch:49 step:193545[D loss: 0.999894] [G loss: 1.000300]
epoch:49 step:193550[D loss: 0.999888] [G loss: 1.000316]
epoch:49 step:193555[D loss: 1.000224] [G loss: 0.999944]
epoch:49 step:193560[D loss: 0.999994] [G loss: 1.000150]
epoch:49 step:193565[D loss: 0.999934] [G loss: 0.999983]
epoch:49 step:193570[D loss: 1.000088] [G loss: 0.999876]
epoch:49 step:193575[D loss: 1.000150] [G loss: 0.999363]
epoch:49 step:193580[D loss: 0.999809] [G loss: 1.000091]
epoch:49 step:193585[D loss: 0.999910] [G loss: 1.000060]
epoch:49 step:

epoch:49 step:194215[D loss: 0.999547] [G loss: 1.000936]
epoch:49 step:194220[D loss: 1.000223] [G loss: 1.000238]
epoch:49 step:194225[D loss: 0.999705] [G loss: 1.001373]
epoch:49 step:194230[D loss: 0.999590] [G loss: 1.001021]
epoch:49 step:194235[D loss: 0.999749] [G loss: 1.000548]
epoch:49 step:194240[D loss: 0.999993] [G loss: 1.000235]
epoch:49 step:194245[D loss: 0.999916] [G loss: 1.000277]
epoch:49 step:194250[D loss: 0.999935] [G loss: 1.000195]
epoch:49 step:194255[D loss: 1.000148] [G loss: 0.999682]
epoch:49 step:194260[D loss: 0.999980] [G loss: 0.999890]
epoch:49 step:194265[D loss: 1.000526] [G loss: 0.998964]
epoch:49 step:194270[D loss: 1.000856] [G loss: 0.998885]
epoch:49 step:194275[D loss: 1.000565] [G loss: 0.999528]
epoch:49 step:194280[D loss: 1.000441] [G loss: 0.999544]
epoch:49 step:194285[D loss: 1.000677] [G loss: 0.999467]
epoch:49 step:194290[D loss: 1.001012] [G loss: 0.998555]
epoch:49 step:194295[D loss: 1.000966] [G loss: 0.998722]
epoch:49 step:

epoch:49 step:194925[D loss: 0.999948] [G loss: 1.000020]
epoch:49 step:194930[D loss: 1.000071] [G loss: 0.999787]
epoch:49 step:194935[D loss: 0.999998] [G loss: 0.999818]
epoch:49 step:194940[D loss: 0.999821] [G loss: 1.000214]
epoch:49 step:194945[D loss: 0.999804] [G loss: 1.000217]
epoch:49 step:194950[D loss: 1.000027] [G loss: 1.000039]
epoch:49 step:194955[D loss: 1.000337] [G loss: 0.999671]
epoch:49 step:194960[D loss: 0.999896] [G loss: 1.000008]
epoch:49 step:194965[D loss: 0.999959] [G loss: 1.000052]
epoch:49 step:194970[D loss: 1.000049] [G loss: 1.000212]
epoch:49 step:194975[D loss: 0.999752] [G loss: 1.000728]
epoch:49 step:194980[D loss: 0.999910] [G loss: 1.000059]
epoch:49 step:194985[D loss: 1.000012] [G loss: 0.999965]
epoch:49 step:194990[D loss: 1.000053] [G loss: 0.999726]
epoch:49 step:194995[D loss: 1.000113] [G loss: 0.999636]
epoch:49 step:195000[D loss: 1.000017] [G loss: 0.999770]
epoch:49 step:195005[D loss: 1.000271] [G loss: 0.999261]
epoch:49 step: